# Modeling

## Loading and Cleaning Datasets

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

2023-03-07 12:44:09.216521: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-07 12:44:10.075134: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-07 12:44:10.075181: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-07 12:44:12.564887: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [5]:
def load_images(folder):
    """
    Enter a folder directory with images to load.
    """
    datagen = ImageDataGenerator(rescale = 1. / 255)
    images = datagen.flow_from_directory(folder,
                                         target_size = (224, 224),
                                         color_mode = "rgb",
                                         batch_size = 32,
                                         class_mode = "categorical")
    return images

In [6]:
def train_val_test_generator(root):
    """
    Generate the train, validation, and test batches.
    """
    #the directories need to be updated once we have the data in the cloud
    #root will be updated to folder
    root = "/home/emilyma/code/FlorenceBoutin/GI_disease_detection/raw_data"
    train_directory = os.path.join(root, "train")
    val_directory = os.path.join(root, "val")
    test_directory = os.path.join(root, "test")

    train_dataset = load_images(train_directory)
    val_dataset = load_images(val_directory)
    test_dataset = load_images(test_directory)

    return train_dataset, val_dataset, test_dataset

In [7]:
train_dataset, val_dataset, test_dataset = train_val_test_generator("/home/emilyma/code/FlorenceBoutin/GI_disease_detection/raw_data")

Found 3200 images belonging to 4 classes.
Found 2000 images belonging to 4 classes.
Found 800 images belonging to 4 classes.


## Modeling

In [8]:
from tensorflow.keras import models
from tensorflow.keras import Sequential, layers, regularizers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import metrics

In [9]:
# reg_l1_l2 = regularizers.l1_l2(l1 = 0.005, l2 = 0.0005)

In [10]:
es = EarlyStopping(patience = 2, restore_best_weights = True)

In [11]:
model = Sequential()
model.add(layers.Conv2D(128, kernel_size = (3, 3), input_shape = (224, 224, 3), activation = "relu", padding = "same"))
model.add(layers.Conv2D(64, kernel_size = (3, 3), activation = "relu"))
model.add(layers.MaxPool2D(pool_size = (2, 2)))
model.add(layers.Dropout(rate = 0.2))
model.add(layers.Conv2D(32, kernel_size = (2, 2), activation = "relu"))
model.add(layers.MaxPool2D(pool_size = (2, 2)))
model.add(layers.Dropout(rate = 0.2))
model.add(layers.Conv2D(16, kernel_size = (2, 2), activation = "relu"))
model.add(layers.MaxPool2D(pool_size = (2, 2)))
model.add(layers.Dropout(rate = 0.2))
model.add(layers.Flatten())
# model.add(layers.Dense(15, activation = "relu", activity_regularizer = reg_l1_l2))
model.add(layers.Dense(8, activation = "relu"))
model.add(layers.Dense(4, activation = "softmax"))

2023-03-07 12:44:37.186605: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-07 12:44:37.187279: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-07 12:44:37.187710: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-FEC965N): /proc/driver/nvidia/version does not exist
2023-03-07 12:44:37.191596: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 128)     3584      
                                                                 
 conv2d_1 (Conv2D)           (None, 222, 222, 64)      73792     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 64)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 111, 111, 64)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 110, 110, 32)      8224      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 55, 55, 32)       0         
 2D)                                                    

In [13]:
model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = [metrics.Accuracy(), metrics.Recall()])

In [ ]:
model.fit(train_dataset, validation_data = val_dataset, epochs = 10, callbacks = [es])

Epoch 1/10


2023-03-07 12:44:44.317640: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 822083584 exceeds 10% of free system memory.
2023-03-07 12:44:44.699960: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 403734528 exceeds 10% of free system memory.
2023-03-07 12:44:47.881632: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 403734528 exceeds 10% of free system memory.
2023-03-07 12:44:48.583062: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 822083584 exceeds 10% of free system memory.


  1/100 [..............................] - ETA: 30:42 - loss: 1.3737 - accuracy: 0.0000e+00 - recall: 0.0000e+00

2023-03-07 12:45:01.430757: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 822083584 exceeds 10% of free system memory.


 17/100 [====>.........................] - ETA: 17:51 - loss: 1.2293 - accuracy: 0.0000e+00 - recall: 0.1562

In [19]:
model.evaluate(test_dataset)

25/25 [==============================] - 10s 400ms/step - loss: 1.1288 - accuracy: 0.0000e+00 - recall_1: 0.2800


[1.1287957429885864, 0.0, 0.2800000011920929]

In [22]:
model_3 = Sequential()
model_3.add(layers.Conv2D(64, kernel_size=(3,3), input_shape=(224, 224, 3), activation='relu', padding='same'))
model_3.add(layers.MaxPool2D(pool_size=(3,3)))
model_3.add(layers.Dropout(rate=0.3))
model_3.add(layers.Conv2D(32, kernel_size=(2,2), activation='relu'))
model_3.add(layers.MaxPool2D(pool_size=(2,2)))
model_3.add(layers.Conv2D(16, kernel_size=(2,2), activation='relu'))
model_3.add(layers.MaxPool2D(pool_size=(2,2)))
model_3.add(layers.Flatten())
model_3.add(layers.Dense(15, activation='relu'))
model_3.add(layers.Dropout(rate=0.3))
model_3.add(layers.Dense(5, activation='relu'))
model_3.add(layers.Dense(4, activation='softmax'))

In [23]:
model_3.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 224, 224, 64)      1792      
                                                                 
                                                                 
 g2D)                                                            
                                                                 
 dropout_8 (Dropout)         (None, 74, 74, 64)        0         
                                                                 
 conv2d_13 (Conv2D)          (None, 73, 73, 32)        8224      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 36, 36, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 35, 35, 16)       

In [24]:
model_3.compile(loss='categorical_crossentropy',
               optimizer='adam',
               metrics=[metrics.Accuracy(), metrics.Recall()])

In [25]:
model_3.fit(train_dataset, validation_data = val_dataset, epochs = 10, callbacks = [es])

Epoch 1/10


2023-03-07 11:44:13.593006: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 411041792 exceeds 10% of free system memory.
2023-03-07 11:44:14.106467: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 411041792 exceeds 10% of free system memory.


  1/100 [..............................] - ETA: 3:38 - loss: 1.3842 - accuracy: 0.0000e+00 - recall: 0.0000e+00

2023-03-07 11:44:14.569127: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 411041792 exceeds 10% of free system memory.
2023-03-07 11:44:15.042432: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 411041792 exceeds 10% of free system memory.


  2/100 [..............................] - ETA: 1:45 - loss: 1.3763 - accuracy: 0.0000e+00 - recall: 0.0000e+00

2023-03-07 11:44:15.649305: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 411041792 exceeds 10% of free system memory.


100/100 [==============================] - 155s 2s/step - loss: 0.9652 - accuracy: 0.0000e+00 - recall: 0.3016 - val_loss: 0.8190 - val_accuracy: 0.0000e+00 - val_recall: 0.4700
Epoch 2/10
100/100 [==============================] - 150s 2s/step - loss: 0.6682 - accuracy: 0.0000e+00 - recall: 0.4897 - val_loss: 0.6833 - val_accuracy: 0.0000e+00 - val_recall: 0.5720
Epoch 3/10
100/100 [==============================] - 162s 2s/step - loss: 0.5004 - accuracy: 3.1250e-04 - recall: 0.7456 - val_loss: 0.5189 - val_accuracy: 0.0000e+00 - val_recall: 0.7100
Epoch 4/10
100/100 [==============================] - 156s 2s/step - loss: 0.3364 - accuracy: 0.0010 - recall: 0.8238 - val_loss: 0.5221 - val_accuracy: 0.0000e+00 - val_recall: 0.7100
Epoch 5/10
100/100 [==============================] - 144s 1s/step - loss: 0.3432 - accuracy: 0.0037 - recall: 0.8203 - val_loss: 0.8757 - val_accuracy: 0.0000e+00 - val_recall: 0.6880


In [26]:
model_3.evaluate(test_dataset)

25/25 [==============================] - 12s 488ms/step - loss: 0.4699 - accuracy: 0.0000e+00 - recall: 0.7300


[0.4698968827724457, 0.0, 0.7300000190734863]